In [1]:


from importlib import reload

import utility_functions
utility_functions = reload(utility_functions)
from utility_functions import *



In [3]:


chrome_path = find_chrome_executable()



In [184]:
tile_frame = pd.DataFrame(tile_data)
tile_frame.to_csv("tile_data.csv", index=False)

In [7]:
tiles = pd.read_csv("tile_data.csv")

In [8]:
distance_mask = tiles["distance"] < 50
oasis_mask = tiles["type"] == "Unoccupied oasis"
resource_mask = True#tiles["resources"] != "Crop"

targets = tiles[distance_mask & oasis_mask & resource_mask]
targets["coordinates"] = targets["coordinates"].apply(lambda s: tuple(int(n) for n in s.strip("()").split(",")))

In [9]:
targets = targets[["index", "coordinates", "distance"]]
targets['last_raided'] = round(time.time(), 0)
targets['next_check'] = round(time.time(), 0)

In [10]:
len(targets)

564

In [12]:
targets.to_csv("oases.csv", index=False)

In [13]:
diag = ensure_cdp_chrome_running(
        chrome_path=chrome_path,           # or r"C:\Program Files\Google\Chrome\Application\chrome.exe"
        remote_port=9222,
        user_data_dir=os.path.join(os.path.expanduser("~"), ".cdp_chrome_profile"),
        extra_args=None,            # e.g., ["--disable-extensions"]
        timeout_secs=25.0,
        kill_if_unreachable=True,
    )



with open("..\diag_pid.txt", "w") as f:
    f.write(str(diag['pid']))
    f.close()

In [14]:
diag['pid']

74400

In [ ]:
while True:

    period_user = get_user_for_period(period_allocations)

    if period_user is None:
        time.sleep(random.randint(5,30) * random.randint(30,60))
        continue
    else:

        if get_current_account() != period_user:
            
            login_to_account(diag, period_user)

    imported_targets = pd.read_csv('oases.csv')
    imported_targets.sort_values('last_raided', inplace=True)

    filtered_imported_targets = imported_targets[imported_targets["distance"] <= RAIDING_RADIUS_FROM_TERANA]


    ### initial troop count
    
    try:
        ensure_navigate(diag, "send_troops")
        overview_page_html = get_outer_html_from_diag(diag)
        troops = analyze_overview_page(overview_page_html)

        if len(troops) == 0:
            time.sleep(random.randint(30,60) * random.randint(1,3))
            continue


        for name, oasis in filtered_imported_targets.iterrows():

            x,y = eval(oasis['coordinates'])
            
            if distance_between_points(village_coords['terana'], (x, y)) > RAIDING_RADIUS_FROM_TERANA:
                continue

            if (datetime.now().time().hour in range(0,4)) or (datetime.now().time().hour in range(12,16)):
                if distance_between_points(village_coords['terana'], (x, y)) > 14:
                    continue


            if (time.time() < oasis['next_check']):
                # print(f"skipping {oasis['coordinates']}")
                continue
            
            
            tile_html = get_tile_info_html_from_diag(diag, x, y)
            oasis_details = parse_tile_details(tile_html)

            if oasis_details['type'] != 'Unoccupied oasis':
                imported_targets.loc[name, 'next_check'] = time.time() + 3600*24*7
                imported_targets.to_csv("oases.csv", index=False)
                continue

            if oasis_details['troops'] is not None:
                imported_targets.loc[name, 'next_check'] = time.time() + 30*60
                imported_targets.to_csv("oases.csv", index=False)
                continue

            # send_troops(x,y, village_id, troop_sub_dict, cookie_dict)
            if send_oasis_raid(diag):
                imported_targets.loc[name, 'last_raided'] = round(time.time(), 0)
                imported_targets.to_csv("oases.csv", index=False)
                time.sleep(random.uniform(0.0,1.0))
            else:
                print("out of troops. waiting.")
                break
    except:
        diag = ensure_cdp_chrome_running(
        chrome_path=chrome_path,           # or r"C:\Program Files\Google\Chrome\Application\chrome.exe"
        remote_port=9222,
        user_data_dir=os.path.join(os.path.expanduser("~"), ".cdp_chrome_profile"),
        extra_args=None,            # e.g., ["--disable-extensions"]
        timeout_secs=25.0,
        kill_if_unreachable=True,
    )
        
    # time.sleep(random.randint(30,60) * random.randint(1,3))

logged in to NobelSword
out of troops. waiting.


In [42]:
tile_html = get_outer_html_from_diag(diag)
oasis_details = parse_tile_details(tile_html)

{'index': 'tile_66_-17',
 'coordinates': (66, -17),
 'type': 'Unoccupied oasis',
 'owner': 'nature',
 'troops': [{'unit': 'Spider', 'count': 1}],
 'distance': 15.5,
 'resources': 'Clay'}

In [100]:
import requests

def simulate_combat(attacker_list, animal_list):
    headers = {
        'Accept': 'application/json',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Content-type': 'application/x-www-form-urlencoded',
        'Origin': 'http://travian.kirilloid.ru',
        'Referer': 'http://travian.kirilloid.ru/warsim2.php',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36',
        # 'Cookie': 'dpi=96; _ga=GA1.2.849942373.1760188930; _gid=GA1.2.1392563283.1760188930',
    }

    base_data_list = eval('[{},{"r":3,"u":[0,0,0,0,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"side":"def"},{"r":2,"R":1,"u":[0,0,0,0,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"b":[0,0],"side":"off"}]')
    base_data_list[1]["u"] = animal_list
    base_data_list[2]["u"] = attacker_list
    
    # return str(base_data_list)
    data = {
        'data': str(base_data_list).replace(" ", "").replace("'", "\""),
        'mode': '9',
    }

    response = requests.post('http://travian.kirilloid.ru/awar2.php', headers=headers, data=data, verify=False)

    if response.status_code == 200:
        if response.json()[0]["type"] == "error":
            return 1
        return response.json()[1]['losses'][0]
    else:
        return 1

In [70]:
response.status_code

200

In [40]:
response.json()

[{'type': 'def', 'pob': 0, 'units': {'3': [0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0]}},
 {'type': 'off',
  'race': 2,
  'units': [0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
  'trapped': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'revived': [],
  'losses': [0.7128371886045082, 0.28716281139549177],
  'hero_health': None,
  'dead_heroes': {'0': 0,
   '1': 0,
   '2': 0,
   '5': 0,
   '6': 0,
   '7': 0,
   '8': 0,
   '3': 0},
  'info': []},
 {'type': 'link', 'id': 'BgQImAEFFsDwEA'},
 {'type': 'warns', 'mask': 0}]

In [44]:
list_of_animals = ['Rat', 'Spider', 'Snake', 'Bat', 'Wild Boar', 'Wolf', 'Bear', 'Crocodile', 'Tiger', 'Elephant']
list_of_gaul_trooops = ['phalanx', 'swordsman', 'pathfinder', 'thunder', 'druid', 'haeduan', 'ram', 'treb', 'cheif', 'settler']

In [ ]:
data_list = eval('[{},{"r":3,"u":[0,0,0,0,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"side":"def"},{"r":2,"R":1,"u":[0,0,0,0,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"b":[0,0],"side":"off"}]')

In [ ]:
data_list[2]["u"] = [0, 0, 0, 3, 0, 0, 0, 0, 0, 0]

[{},
 {'r': 3,
  'u': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  'U': [0, 0, 0, 0, 0, 0, 0, 0],
  'side': 'def'},
 {'r': 2,
  'R': 1,
  'u': [0, 0, 0, 3, 0, 0, 0, 0, 0, 0],
  'U': [0, 0, 0, 0, 0, 0, 0, 0],
  'b': [0, 0],
  'side': 'off'}]

In [ ]:
# def get_simulator_attacker_loss(oasis_details, attacking_troops_input):


animals = oasis_details['troops']

input_animal_list = np.zeros(shape=(1,10), dtype=int).tolist()[0]
for animal_dict in animals:
    input_animal_list[list_of_animals.index(animal_dict["unit"])] = animal_dict["count"]

attacker_list = [0, 0, 0, 3, 0, 0, 0, 0, 0, 0]
response = simulate_combat(attacker_list, input_animal_list)

In [102]:
response

0.06370669049627474

In [91]:
response == '[{},{"r":3,"u":[8,3,0,0,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"side":"def"},{"r":2,"R":1,"u":[0,0,0,3,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"b":[0,0],"side":"off"}]'

False

In [95]:
print(response.replace(" ", "").replace("'", "\""))
print('[{},{"r":3,"u":[0,1,0,0,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"side":"def"},{"r":2,"R":1,"u":[0,0,0,3,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"b":[0,0],"side":"off"}]')

[{},{"r":3,"u":[0,1,0,0,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"side":"def"},{"r":2,"R":1,"u":[0,0,0,3,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"b":[0,0],"side":"off"}]
[{},{"r":3,"u":[0,1,0,0,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"side":"def"},{"r":2,"R":1,"u":[0,0,0,3,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"b":[0,0],"side":"off"}]


In [96]:
response.replace(" ", "").replace("'", "\"") == '[{},{"r":3,"u":[0,1,0,0,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"side":"def"},{"r":2,"R":1,"u":[0,0,0,3,0,0,0,0,0,0],"U":[0,0,0,0,0,0,0,0],"b":[0,0],"side":"off"}]'

True